## 反距离

### 使用示例数据创建反距离权的空间权重矩阵



In [2]:
import geopandas as gpd
import pandas as pd
import numpy as np

# 读取Shapefile文件
shp_path = r'示例数据/新建文件夹/CTCI_墨卡托投影.shp'
gdf = gpd.read_file(shp_path)

# 提取所需的字段（ID, X, Y, Z）
df = gdf[['ID', 'X', 'Y', 'Z']].copy()

# 设置距离阈值L0
L0 = 100000

# 创建一个空的空间权重矩阵
spatial_weights = np.zeros((len(df), len(df)))

# 计算空间权重矩阵
for i in range(len(df)):
    for j in range(len(df)):
        # 计算两要素间的实际距离
        distance = np.sqrt((df['X'][i] - df['X'][j])**2 + (df['Y'][i] - df['Y'][j])**2 + (df['Z'][i] - df['Z'][j])**2)
        
        # 判断距离是否小于阈值，并应用反距离权重（距离的倒数，加上一个小数值防止除以0错误）
        if distance <= L0:
            spatial_weights[i, j] = 1 / (distance + 1e-9)

# 将空间权重矩阵保存到DataFrame中
spatial_weights_df = pd.DataFrame(spatial_weights)

# 保存空间权重矩阵为CSV文件
output_file_path = r'示例数据/spatial_weights_matrix反距离权.csv'
spatial_weights_df.to_csv(output_file_path)

### 使用黄淮海平原火点数据，数据过大，不能正常运行，尝试使用kd树以及稀疏矩阵等优化算法，但是运行400分钟仍未得出结果，

In [4]:
import geopandas as gpd
import pandas as pd
import numpy as np

# 读取Shapefile文件
shp_path = r'置信度30以上的黄淮海平原区火点_墨卡托/置信度30以上的黄淮海平原火点_墨卡托.shp'
gdf = gpd.read_file(shp_path)

# 提取所需的字段（X, Y, Z）
df = gdf[['X', 'Y', 'Z']].copy()

# 创建ID序列
ids = range(len(df))

# 设置距离阈值L0
L0 = 100000

# 创建一个空的空间权重矩阵
spatial_weights = np.zeros((len(df), len(df)))

# 计算空间权重矩阵
for i in ids:
    for j in ids:
        # 计算两要素间的实际距离
        distance = np.sqrt((df['X'][i] - df['X'][j])**2 + (df['Y'][i] - df['Y'][j])**2 + (df['Z'][i] - df['Z'][j])**2)
        
        # 判断距离是否小于阈值，并应用反距离权重（距离的倒数，加上一个小数值防止除以0错误）
        if distance <= L0:
            spatial_weights[i, j] = 1 / (distance + 1e-9)

# 将空间权重矩阵保存到DataFrame中，添加ID作为行索引和列索引
spatial_weights_df = pd.DataFrame(spatial_weights, index=ids, columns=ids)

# 保存空间权重矩阵为CSV文件
output_file_path = r'示例数据/spatial_weights_matrix黄反距离权1.csv'
spatial_weights_df.to_csv(output_file_path, header=True)

MemoryError: Unable to allocate 141. GiB for an array with shape (137407, 137407) and data type float64

以下是使用kd树的代码，不知道是否正确，因为跑了很长时间，没有结果。


In [ ]:
import geopandas as gpd
from scipy.spatial import KDTree
import pandas as pd
from scipy.sparse import csr_matrix
import numpy as np

# 读取Shapefile文件
shp_path = r'置信度30以上的黄淮海平原区火点_墨卡托/置信度30以上的黄淮海平原火点_墨卡托.shp'
gdf = gpd.read_file(shp_path)

# 确保ID字段存在，如果没有则生成
if 'ID' not in gdf.columns:
    gdf['ID'] = range(1, len(gdf) + 1)

# 提取平面坐标
xy_coords = np.vstack([gdf['X'].values, gdf['Y'].values]).T

# 使用KDTree建立索引
tree = KDTree(xy_coords)

# 设置距离衰减参数k（例如：k=2代表平方反比距离权重）
k = 2
# 设置平滑因子（可选，避免除以零）
epsilon = 0.0001

# 计算并填充空间权重
num_points = len(gdf)
spatial_weights = csr_matrix((num_points, num_points), dtype=np.float64)

# 遍历所有点
for i in range(num_points):
    point_i = xy_coords[i]
    z_i = gdf.loc[i, 'Z']
    
    # 找到距离点i一定范围内的邻居点
    # 这里假设有一个合理的距离阈值（threshold），可以根据实际情况设置
    threshold=10000
    neighbors = tree.query_ball_point(point_i, threshold)
    
    for neighbor_id in neighbors:
        point_j = xy_coords[neighbor_id]
        z_j = gdf.loc[neighbor_id, 'Z']
        
        # 计算三维欧氏距离
        distance_3d = ((point_i[0]-point_j[0])**2 + (point_i[1]-point_j[1])**2 + (z_i-z_j)**2)**0.5
        
        # 计算权重
        weight = 1 / distance_3d**k + epsilon
        
        spatial_weights[i, neighbor_id] = weight
        spatial_weights[neighbor_id, i] = weight  # 权重是对称的

# 将稀疏矩阵转换为DataFrame
indices = gdf['ID'].values
sparse_weights_df = pd.DataFrame.sparse.from_spmatrix(spatial_weights, index=indices, columns=indices)
 
dense_weights = spatial_weights.todense()

# 创建一个 dense DataFrame
dense_weights_df = pd.DataFrame(dense_weights, index=indices, columns=indices)

# 写入CSV文件
output_file_path = r'spatial_weights_matrix1.csv'
dense_weights_df.to_csv(output_file_path, header=True, index_label='ID')

### 阈值权
没有做多调整，


In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np

# 读取Shapefile文件
shp_path = r'示例数据/新建文件夹/CTCI_墨卡托投影.shp'
gdf = gpd.read_file(shp_path)

# 提取所需的字段（ID, X, Y, Z）
df = gdf[['ID', 'X', 'Y', 'Z']].copy()

# 设置距离阈值L0
L0 = 100000

# 创建一个空的空间权重矩阵
spatial_weights = np.zeros((len(df), len(df)))

# 计算空间权重矩阵
for i in range(len(df)):
    for j in range(len(df)):
        # 计算两要素间的实际距离
        distance = np.sqrt((df['X'][i] - df['X'][j])**2 + (df['Y'][i] - df['Y'][j])**2 + (df['Z'][i] - df['Z'][j])**2)
        
        # 判断距离是否小于阈值
        if distance <= L0:
            spatial_weights[i, j] = 1

# 将空间权重矩阵保存到DataFrame中
spatial_weights_df = pd.DataFrame(spatial_weights)

# 保存空间权重矩阵为CSV文件
output_file_path = r'示例数据/spatial_weights_matrix阈值权.csv'
spatial_weights_df.to_csv(output_file_path)
